In [1]:
import pandas as pd

## Read

In [2]:
user_behaviour = pd.read_csv('../data/processed/user_behaviour.csv')
user_behaviour.head()

,profile_id,event,time,portfolio_id,amount,transcript_reward,reward,channels,difficulty,duration,offer_type,gender,age,income,become_member_on_date
0,78afa995795e4d85b5d9ceeca43f5fef,offer_received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,5.0,web,5.0,7.0,bogo,F,75,100000.0,2017-05-09
1,78afa995795e4d85b5d9ceeca43f5fef,offer_received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,5.0,email,5.0,7.0,bogo,F,75,100000.0,2017-05-09
2,78afa995795e4d85b5d9ceeca43f5fef,offer_received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,5.0,mobile,5.0,7.0,bogo,F,75,100000.0,2017-05-09
3,a03223e636434f42ac4c3df47e8bac43,offer_received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,5.0,web,20.0,10.0,discount,NaN,118,NaN,2017-08-04
4,a03223e636434f42ac4c3df47e8bac43,offer_received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,5.0,email,20.0,10.0,discount,NaN,118,NaN,2017-08-04


## Select required records

In [3]:
user_behaviour = user_behaviour[user_behaviour["event"] == "offer_viewed"]

## Add new columns

In [4]:
# Change the way the porftolio id is represented
portfolio_summary = user_behaviour['portfolio_id'].value_counts().reset_index()
portfolio_summary.columns = ['portfolio_id', 'count']
portfolio_summary['percentage'] = (portfolio_summary['count'] / portfolio_summary['count'].sum()) * 100
portfolio_summary.dropna(inplace=True)
portfolio_summary['offer'] = 'offer_' + portfolio_summary.index.astype(str)
portfolio_summary

,portfolio_id,count,percentage,offer
0,2298d6c36e964ae4a3e7e9706d1fb8c2,29348,14.693248,offer_0
1,fafdcd668e3743c1bb461111dcafc2a4,29308,14.673222,offer_1
2,4d5c57ea9a6940dd891ad53e9dbe8da0,29192,14.615146,offer_2
3,f19421c1d4aa40978ebb69ca19b0e20d,29056,14.547057,offer_3
4,ae264e3637204a6fb9bb56bc8210ddfd,20148,10.087214,offer_4
5,5a8bc65990b245e5a138643cd4eb9837,20061,10.043657,offer_5
6,9b98b8c7a33c4b65b9aebfe6a799e6d9,12513,6.264707,offer_6
7,3f207df678b143eea3cee63160fa8bed,12432,6.224154,offer_7
8,2906b810c7d4411798c6938adc9daaa5,12354,6.185102,offer_8
9,0b1e1539f2cc45b7b9fa7c272da2e1d7,5326,2.666493,offer_9


In [5]:
portfolio_summary.to_csv('../data/processed/portfolio_summary.csv', index=False)

In [6]:
# Merge the offer column from portfolio_summary into user_behaviour
user_behaviour = user_behaviour.merge(
    portfolio_summary[['portfolio_id', 'offer']], 
    on='portfolio_id', 
    how='left'
)

# Show the first few rows to verify the merge
user_behaviour[['profile_id', 'portfolio_id', 'offer']].head()

,profile_id,portfolio_id,offer
0,389bc3fa690240e798340f5a15918d5c,f19421c1d4aa40978ebb69ca19b0e20d,offer_3
1,389bc3fa690240e798340f5a15918d5c,f19421c1d4aa40978ebb69ca19b0e20d,offer_3
2,389bc3fa690240e798340f5a15918d5c,f19421c1d4aa40978ebb69ca19b0e20d,offer_3
3,389bc3fa690240e798340f5a15918d5c,f19421c1d4aa40978ebb69ca19b0e20d,offer_3
4,d1ede868e29245ea91818a903fec04c6,5a8bc65990b245e5a138643cd4eb9837,offer_5


In [7]:
user_behaviour['become_member_on_date'] = pd.to_datetime(user_behaviour['become_member_on_date'])
user_behaviour['become_member_on_year'] = user_behaviour['become_member_on_date'].dt.year.astype(str)
user_behaviour[['profile_id', 'offer', 'become_member_on_date','become_member_on_year']].head()

,profile_id,offer,become_member_on_date,become_member_on_year
0,389bc3fa690240e798340f5a15918d5c,offer_3,2018-02-09,2018
1,389bc3fa690240e798340f5a15918d5c,offer_3,2018-02-09,2018
2,389bc3fa690240e798340f5a15918d5c,offer_3,2018-02-09,2018
3,389bc3fa690240e798340f5a15918d5c,offer_3,2018-02-09,2018
4,d1ede868e29245ea91818a903fec04c6,offer_5,2017-09-16,2017


## Transform categorical variables to numerical variables

In [8]:
# Create dummy variables for categorical columns
## From transcript
event_dummies = pd.get_dummies(user_behaviour['event'], prefix='event')
## From portfolio
offer_dummies = pd.get_dummies(user_behaviour['offer'])
offer_type_dummies = pd.get_dummies(user_behaviour['offer_type'], prefix='offer_type')
channels_dummies = pd.get_dummies(user_behaviour['channels'], prefix='channels')
## From profile
gender_dummies = pd.get_dummies(user_behaviour['gender'], prefix='gender')
become_member_on_year_dummies = pd.get_dummies(user_behaviour['become_member_on_year'], prefix='become_member_on_year')


# Concatenate all dummy variables with the original dataframe
user_behaviour = pd.concat([
    user_behaviour,
    event_dummies,
    offer_dummies,
    offer_type_dummies,
    channels_dummies,
    gender_dummies,
    become_member_on_year_dummies    
], axis=1)

user_behaviour.head()

,profile_id,event,time,portfolio_id,amount,transcript_reward,reward,channels,difficulty,duration,...,channels_web,gender_F,gender_M,gender_O,become_member_on_year_2013,become_member_on_year_2014,become_member_on_year_2015,become_member_on_year_2016,become_member_on_year_2017,become_member_on_year_2018
0,389bc3fa690240e798340f5a15918d5c,offer_viewed,0,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN,5.0,web,5.0,5.0,...,True,False,True,False,False,False,False,False,False,True
1,389bc3fa690240e798340f5a15918d5c,offer_viewed,0,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN,5.0,email,5.0,5.0,...,False,False,True,False,False,False,False,False,False,True
2,389bc3fa690240e798340f5a15918d5c,offer_viewed,0,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN,5.0,mobile,5.0,5.0,...,False,False,True,False,False,False,False,False,False,True
3,389bc3fa690240e798340f5a15918d5c,offer_viewed,0,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN,5.0,social,5.0,5.0,...,False,False,True,False,False,False,False,False,False,True
4,d1ede868e29245ea91818a903fec04c6,offer_viewed,0,5a8bc65990b245e5a138643cd4eb9837,NaN,NaN,0.0,email,0.0,3.0,...,False,False,False,True,False,False,False,False,True,False


## Remove unneded variables

In [9]:
# Drop the original categorical columns
user_behaviour.drop(['portfolio_id','event','offer','offer_type','channels','gender','become_member_on_year','become_member_on_date'], axis=1, inplace=True)
user_behaviour.head()

,profile_id,time,amount,transcript_reward,reward,difficulty,duration,age,income,event_offer_viewed,...,channels_web,gender_F,gender_M,gender_O,become_member_on_year_2013,become_member_on_year_2014,become_member_on_year_2015,become_member_on_year_2016,become_member_on_year_2017,become_member_on_year_2018
0,389bc3fa690240e798340f5a15918d5c,0,NaN,NaN,5.0,5.0,5.0,65,53000.0,True,...,True,False,True,False,False,False,False,False,False,True
1,389bc3fa690240e798340f5a15918d5c,0,NaN,NaN,5.0,5.0,5.0,65,53000.0,True,...,False,False,True,False,False,False,False,False,False,True
2,389bc3fa690240e798340f5a15918d5c,0,NaN,NaN,5.0,5.0,5.0,65,53000.0,True,...,False,False,True,False,False,False,False,False,False,True
3,389bc3fa690240e798340f5a15918d5c,0,NaN,NaN,5.0,5.0,5.0,65,53000.0,True,...,False,False,True,False,False,False,False,False,False,True
4,d1ede868e29245ea91818a903fec04c6,0,NaN,NaN,0.0,0.0,3.0,53,52000.0,True,...,False,False,False,True,False,False,False,False,True,False


## Aggregate

In [10]:
# List of columns to sum (all dummy columns for offers, offer types, events, and channels)
sum_columns = [
    col for col in user_behaviour.columns 
    if col.startswith(('reward','difficulty','duration','transcript_reward','time', 'amount', 'offer_', 'event_', 'channels_'))
]

# Get remaining columns (excluding the ones we're summing)
group_columns = [col for col in user_behaviour.columns if col not in sum_columns]
print(group_columns)

['profile_id', 'age', 'income', 'gender_F', 'gender_M', 'gender_O', 'become_member_on_year_2013', 'become_member_on_year_2014', 'become_member_on_year_2015', 'become_member_on_year_2016', 'become_member_on_year_2017', 'become_member_on_year_2018']


In [11]:
# Group by remaining columns and sum the dummy columns
user_behaviour_aggregated = user_behaviour.groupby(group_columns)[sum_columns].sum().reset_index()
user_behaviour_aggregated.head()

,profile_id,age,income,gender_F,gender_M,gender_O,become_member_on_year_2013,become_member_on_year_2014,become_member_on_year_2015,become_member_on_year_2016,...,offer_7,offer_8,offer_9,offer_type_bogo,offer_type_discount,offer_type_informational,channels_email,channels_mobile,channels_social,channels_web
0,0009655768c64bdeb2e877511632db8f,33,72000.0,False,True,False,False,False,False,False,...,3,0,0,4,4,6,4,4,3,3
1,0011e0d4e6b944f998e987f904e8c1e5,40,57000.0,False,False,True,False,False,False,False,...,3,0,2,3,6,6,5,4,2,4
2,0020c2b971eb4e9188eac86d93036a77,59,90000.0,True,False,False,False,False,False,True,...,0,0,0,4,4,3,3,3,3,2
3,0020ccbbb6d84e358d3414a3ff76cffd,24,60000.0,True,False,False,False,False,False,True,...,0,0,0,7,4,3,4,4,3,3
4,003d66b6608740288d6cc97a6903f4f0,26,73000.0,True,False,False,False,False,False,False,...,3,0,0,0,8,6,4,4,3,3


In [12]:
# Count distinct profile_ids and total rows
distinct_profiles = len(user_behaviour_aggregated['profile_id'].unique())
total_rows = len(user_behaviour_aggregated)

print(f"Number of distinct profiles: {distinct_profiles}")
print(f"Total number of rows: {total_rows}")

Number of distinct profiles: 14675
Total number of rows: 14675


## Save

In [13]:
user_behaviour_aggregated.to_csv('../data/features/user_behaviour_offer_viewed.csv', index=False)